<a href="https://colab.research.google.com/github/radosty/radosty.github.io/blob/main/task9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


prices_df = pd.read_csv('https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/davies_price_data/db_prices.csv')
print(prices_df.sample(5))

items_df = pd.read_csv('https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/davies_price_data/db_item_clean.csv')
print(items_df.head())


          quote_date  shop_code  item_id_raw  region   price indicator_box  \
20762563    201211.0      278.0       410508     9.0   25.00           NaN   
16222580    199501.0       27.0       310107    12.0    0.90           NaN   
22616502    199606.0      244.0       430105     2.0  650.00           NaN   
31348976    199502.0      179.0       510201     3.0  149.00             S   
18575361    200108.0      167.0       310305     7.0    9.95           NaN   

          item_id  
20762563   410508  
16222580   310107  
22616502   430105  
31348976   510201  
18575361   310315  
   item_id                     description  date_quote_s  date_quote_e  n_obs
0   210101    LARGE LOAF-WHITE-SLICED-800G        198802        200401  36039
1   210102  LARGE LOAF-WHITE-UNSLICED-800G        198802        202409  56048
2   210105   LARGE WHOLEMEAL LOAF-UNSLICED        198802        200301  27161
3   210106     SIX BREAD ROLLS-WHITE/BROWN        198802        202409  65710
4   210107     BROWN 

In [5]:
full_df = pd.merge(prices_df, items_df[['item_id', 'description']], on='item_id')
full_df['quote_date'] = pd.to_datetime(full_df['quote_date'], format='%Y%m')
#this will tell me the ids of products of interest
items_df[items_df.description.str.contains('cheese', case=False)].sort_values(by=["date_quote_e", "date_quote_s"])

,item_id,description,date_quote_s,date_quote_e,n_obs
145,211505,CHEESE CHESHIRE-PER KG,199002,199801,16520
148,211508,PROCESSED CHEESE SLICES (10),200202,200401,3057
146,211506,"CHEESE SPREAD, 125g-200g TUB",199802,202409,38157
149,211509,HARD REGIONAL CHEESE PER KG,200402,202409,33022
150,211510,PARMESAN CHEESE-PER KG,200902,202409,23849
151,211511,SOFT CONTINENTAL CHEESE - KG,201202,202409,33019


In [8]:
#select parmesan and soft continental
cheese_ids = [211510, 211511]
#include only rows of cheese_ids
df = full_df[full_df['item_id'].isin(cheese_ids)]
#new column year_month that extracts the year and month from the quote_date column in period format
df['year_month'] = df['quote_date'].dt.to_period('M')
#groups the data by year_month and my 2 cheeses and computes a mean for each
yearanditem_df = df.groupby(['year_month', 'item_id']).agg({'price': 'mean'}).reset_index()
cheese_df = yearanditem_df.pivot(index='year_month', columns='item_id', values='price').reset_index()
cheese_df.rename(columns={211510: 'Parmesan', 211511: 'Soft Continental Cheese'}, inplace=True)
#drop rows with missing values
cheese_df = cheese_df.dropna()
cheese_df.to_csv('cheeseprices.csv', index=False)


<ipython-input-8-34d7e1ddce7d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_month'] = df['quote_date'].dt.to_period('M')
